In [1]:
#Wikipedia scraping for track and artist names of Billboard hits
import requests as rq
from bs4 import BeautifulSoup
import pandas as pd

#Wikipedia has seperate page for each year.
#Iterate through each page
for year in range (1960,2020):
    url_text = str('https://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_' + str(year))
    #Obtain html script of the page
    website_url = rq.get(url_text).text
  
    soup = BeautifulSoup(website_url, 'lxml')
    #print(soup.prettify())
    
    #Data stored in a table and in the html script this is under 'wikitable sortable'
    My_table = soup.find('table', {'class' : 'wikitable sortable'}).tbody
    
    #Each individual row marked by 'tr'
    rows = My_table.find_all('tr')
    
    #Create table columns for data frame
    columns = [v.text.replace('\n', '') for v in rows[0].find_all('th')]
    columns.append('year')
    df = pd.DataFrame(columns = columns)

    #Iterate through each row
    for i in range(1,len(rows)):
        #Track and artist name marked by 'td'
        #Append values to a list 
        tds = rows[i].find_all('td')
        if len(tds) == 3:
            values = [td.text for td in tds]
            values.append(year)
        else:
            values = [i, tds[0].text, tds[1].text, year] 
        #Add list of values to data frame
        df = df.append(pd.Series(values,index=columns), ignore_index=True)
    
    #Store to a csv file
    df.to_csv(r'billboard_data.csv', mode='a', index=False)
